Tables:
- `accounts` - Facebook public profiles with some data on them
- `posts` - posts of these public profiles, you can use the profile_id column to connect to
the accounts table
- `sources_for_followers` - number of followers for these public profiles

In [195]:
import pandas as pd
from sqlalchemy import create_engine

In [196]:
db_params = {
    'host': 'localhost',
    'port': 5432,
    'database': 'postgres',
    'user': 'postgres',
    'password': 'postgres_password'
}

In [197]:
engine = create_engine(f'postgresql://{db_params["user"]}:{db_params["password"]}@{db_params["host"]}/{db_params["database"]}')

In [198]:
csv_files = {
    'accounts': 'datasets/accounts.csv',
    'posts': 'datasets/posts.csv',
    'sources_for_followers': 'datasets/sources_for_followers.csv'
}

In [199]:
for table_name, file_path in csv_files.items():
    df = pd.read_csv(file_path)
    df.to_sql(table_name, engine, if_exists='replace', index=False)

Let's see the info of every table.

In [200]:
for table_name in csv_files.keys():
    query = f"""SELECT *              
               FROM {table_name};"""
    table = pd.read_sql(query, engine)
    print(f'Information about {table_name} table:')
    print(table.info(), '\n')

Information about accounts table:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           30 non-null     int64  
 1   username     28 non-null     object 
 2   full_name    30 non-null     object 
 3   description  28 non-null     object 
 4   is_verified  27 non-null     object 
 5   restricted   0 non-null      object 
 6   _id          30 non-null     int64  
 7   _status      30 non-null     object 
 8   id_alt       3 non-null      float64
dtypes: float64(1), int64(2), object(6)
memory usage: 2.2+ KB
None 

Information about posts table:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1138 entries, 0 to 1137
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              1138 non-null   int64 
 1   created_time    1138 non-null   object
 2   profile_id     

The `accounts` table contains 30 records, with some columns having missing values, and the `restricted` column being entirely empty.
The `posts` table consists of 1138 records, with all fields fully populated.
The `sources_for_followers` table contains 30 records, with all fields fully populated.

We will analyze these tables in more detail later.

Let's check if we can match all records by account id.

In [201]:
query = """SELECT COUNT(*)        
           FROM accounts AS a
           FULL JOIN sources_for_followers AS s
           ON a._id = s._id
           FULL JOIN posts as p
           ON a.id = p.profile_id
           WHERE a.id IS NULL OR s._id is NULL OR p.id IS NULL;"""
print(pd.read_sql(query, engine))

   count
0    120


After merging, we have 120 records where either the account id or post id is missing, or the account ID comes from the `sources_for_followers` table.

Let's see if each account has details about its follower count.

In [202]:
query = """SELECT COUNT(*)              
           FROM accounts
           LEFT JOIN sources_for_followers
           USING(_id)
           WHERE followers_count IS NULL;"""
print(pd.read_sql(query, engine))

   count
0      0


We need to check if we have information on all post authors.

In [203]:
query = """SELECT DISTINCT profile_id       
           FROM posts AS p
           LEFT JOIN accounts AS a
           ON p.profile_id = a.id
           WHERE a.id IS NULL;"""
print(pd.read_sql(query, engine))

         profile_id
0   100064141090703
1  2212856535602171
2   100064478088503


We have three post authors missing from the `accounts` table. However, the `accounts` table includes a column called `id_alt`, so let's check if we can use this column.


In [204]:
query = """SELECT id_alt        
           FROM accounts AS a
           WHERE id_alt IS NOT NULL;"""
print(pd.read_sql(query, engine).astype(int))

             id_alt
0   100064141090703
1  2212856535602171
2   100064478088503


Now we can merge all our data, keeping the result for future analysis.

In [205]:
query = """SELECT p.profile_id, a.*, s.followers_count, p.id AS post_id, p.created_time AS post_created_time, 
                  p.text_original AS post_text, p.comments_count
           FROM accounts AS a
           LEFT JOIN sources_for_followers AS s
           ON a._id = s._id
           LEFT JOIN posts as p
           ON a.id = p.profile_id OR a.id_alt = p.profile_id;"""
accounts_with_posts_data = pd.read_sql(query, engine)

Let's analyze what data we have.

In [206]:
accounts_with_posts_data.head()

,profile_id,id,username,full_name,description,is_verified,restricted,_id,_status,id_alt,followers_count,post_id,post_created_time,post_text,comments_count
0,100063695660293,100063695660293,ukraina.ru,Украина.ру,Информационно-аналитическое издание «Украина.ру»,False,None,378,downloaded,NaN,38000,965760471089655,2023-04-04 12:30:08+00,Emil Berdeli: Gușă a fost singurul dintre jurn...,3
1,100063695660293,100063695660293,ukraina.ru,Украина.ру,Информационно-аналитическое издание «Украина.ру»,False,None,378,downloaded,NaN,38000,979082653090770,2023-04-23 18:15:18+00,Adrian Papahagi anunță pe pagina sa de Faceboo...,36
2,100063695660293,100063695660293,ukraina.ru,Украина.ру,Информационно-аналитическое издание «Украина.ру»,False,None,378,downloaded,NaN,38000,965077954491240,2023-04-03 12:27:58+00,Am vizitat ieri un targ al refugiatilor ucrain...,24
3,100063695660293,100063695660293,ukraina.ru,Украина.ру,Информационно-аналитическое издание «Украина.ру»,False,None,378,downloaded,NaN,38000,964575337874835,2023-04-02 16:06:21+00,Azi se implinesc 19 ani de la aderarea Romanie...,1
4,100063695660293,100063695660293,ukraina.ru,Украина.ру,Информационно-аналитическое издание «Украина.ру»,False,None,378,downloaded,NaN,38000,978371336495235,2023-04-22 15:37:57+00,Adrian Papahagi: Există limite în orice. Acest...,2


During the overview, we observed that the `restricted` column is completely empty, the `_status` column contains the same value in all rows, and the `_id` column does not provide any meaningful information. Therefore, we can remove these columns. We will also remove `id` and `id_alt`, leaving only `profile_i`d as the account identifier.

In [207]:
columns_to_drop = ['_id', 'restricted', '_status', 'id', 'id_alt']
accounts_with_posts_data = accounts_with_posts_data.drop(columns=columns_to_drop)

In [208]:
accounts_with_posts_data.head()

,profile_id,username,full_name,description,is_verified,followers_count,post_id,post_created_time,post_text,comments_count
0,100063695660293,ukraina.ru,Украина.ру,Информационно-аналитическое издание «Украина.ру»,False,38000,965760471089655,2023-04-04 12:30:08+00,Emil Berdeli: Gușă a fost singurul dintre jurn...,3
1,100063695660293,ukraina.ru,Украина.ру,Информационно-аналитическое издание «Украина.ру»,False,38000,979082653090770,2023-04-23 18:15:18+00,Adrian Papahagi anunță pe pagina sa de Faceboo...,36
2,100063695660293,ukraina.ru,Украина.ру,Информационно-аналитическое издание «Украина.ру»,False,38000,965077954491240,2023-04-03 12:27:58+00,Am vizitat ieri un targ al refugiatilor ucrain...,24
3,100063695660293,ukraina.ru,Украина.ру,Информационно-аналитическое издание «Украина.ру»,False,38000,964575337874835,2023-04-02 16:06:21+00,Azi se implinesc 19 ani de la aderarea Romanie...,1
4,100063695660293,ukraina.ru,Украина.ру,Информационно-аналитическое издание «Украина.ру»,False,38000,978371336495235,2023-04-22 15:37:57+00,Adrian Papahagi: Există limite în orice. Acest...,2
